In [12]:
# -*- coding: utf-8 -*-
from __future__ import division, print_function
from gensim.models import Word2Vec
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, Merge
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.models import Sequential
import nltk
from keras.preprocessing.sequence import pad_sequences
from sklearn.cross_validation import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import os
import keras
import kaggle

In [13]:
DATA_DIR = "../data/comp_data"
MODEL_DIR = "../data/models"
WORD2VEC_BIN = "studystack.bin"
qa_BIN = "qa_total.bin"
WORD2VEC_EMBED_SIZE = 300

QA_TRAIN_FILE = "Middle-NDMC-Train.csv"
FLASHCARD_SENTS = "studystack_qa_cleaner_no_qm.txt"

QA_EMBED_SIZE = 64
BATCH_SIZE = 32
NBR_EPOCHS = 20

## extract data

print("Loading and formatting data...")
qapairs = kaggle.get_question_answer_pairs(
    question_file=os.path.join(DATA_DIR, QA_TRAIN_FILE), qaf_file=os.path.join(DATA_DIR, FLASHCARD_SENTS) )
question_maxlen = max([len(qapair[0]) for qapair in qapairs])
answer_maxlen = max([len(qapair[1]) for qapair in qapairs])
seq_maxlen = max([question_maxlen, answer_maxlen])

word2idx = kaggle.build_vocab([], qapairs, [])
vocab_size = len(word2idx) + 1 # include mask character 0

Xq, Xa, Y = kaggle.vectorize_qapairs(qapairs, word2idx, seq_maxlen)
Xqtrain, Xqtest, Xatrain, Xatest, Ytrain, Ytest = \
    train_test_split(Xq, Xa, Y, test_size=0.3, random_state=42)
print(Xqtrain.shape, Xqtest.shape, Xatrain.shape, Xatest.shape, 
      Ytrain.shape, Ytest.shape)

Loading and formatting data...
(319574, 185) (136961, 185) (319574, 185) (136961, 185) (319574, 2) (136961, 2)


In [16]:
print(qapairs[3])

(['Which', 'most', 'accurately', 'describes', 'a', 'circle', '(', 'pie', ')', 'graph', '?'], ['used', 'to', 'describe', 'how', 'part', 'of', 'the', 'data', 'relates', 'to', 'all', 'of', 'the', 'data'], True)


In [17]:
# get embeddings from word2vec
# see https://github.com/fchollet/keras/issues/853
print("Loading Word2Vec model and generating embedding matrix...")
word2vec = Word2Vec.load(os.path.join(DATA_DIR, WORD2VEC_BIN))
embedding_weights = np.zeros((vocab_size, WORD2VEC_EMBED_SIZE))
for word, index in word2idx.items():
    try:
        embedding_weights[index, :] = word2vec[word.lower()]
    except KeyError:
        pass  # keep as zero (not ideal, but what else can we do?)

Loading Word2Vec model and generating embedding matrix...


C:\Users\RAKSHITH_RAO\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:8: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [19]:
print("Building model...")
qenc = Sequential()
qenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   input_length=seq_maxlen,
                   weights=[embedding_weights]))
qenc.add(Bidirectional(LSTM(QA_EMBED_SIZE, return_sequences=True),merge_mode="sum"))
qenc.add(Dropout(0.3))

aenc = Sequential()
aenc.add(Embedding(output_dim=WORD2VEC_EMBED_SIZE, input_dim=vocab_size,
                   input_length=seq_maxlen,
                   weights=[embedding_weights]))
aenc.add(Bidirectional(LSTM(QA_EMBED_SIZE, return_sequences=True),merge_mode="sum"))
aenc.add(Dropout(0.3))


Building model...


ResourceExhaustedError: OOM when allocating tensor with shape[183292,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: embedding_4/embeddings/Assign = Assign[T=DT_FLOAT, use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](embedding_4/embeddings, embedding_4/random_uniform)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'embedding_4/embeddings/Assign', defined at:
  File "C:\Users\RAKSHITH_RAO\Anaconda3\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\RAKSHITH_RAO\Anaconda3\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\RAKSHITH_RAO\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "C:\Users\R

In [11]:
attn = Sequential()
attn.add(Merge([qenc, aenc], mode="dot", dot_axes=[1, 1]))
attn.add(Flatten())
attn.add(Dense((seq_maxlen * QA_EMBED_SIZE)))
attn.add(Reshape((seq_maxlen, QA_EMBED_SIZE)))

model = Sequential()
model.add(Merge([qenc, attn], mode="sum"))
model.add(Flatten())
model.add(Dense(2, activation="softmax"))

model.compile(optimizer="adam", loss="categorical_crossentropy",
              metrics=["accuracy"])

C:\Users\RAKSHITH_RAO\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  from ipykernel import kernelapp as app
C:\Users\RAKSHITH_RAO\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel\__main__.py:8: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [10]:
print("Training...")
checkpoint = ModelCheckpoint(
    filepath=os.path.join(MODEL_DIR, "qa-blstm-fem-attn-best.hdf5"),
    verbose=1, save_best_only=True)
model.fit([Xqtrain, Xatrain], Ytrain, batch_size=BATCH_SIZE,
          nb_epoch=NBR_EPOCHS, validation_split=0.25,
          callbacks=[checkpoint])

Training...


C:\Users\RAKSHITH_RAO\Anaconda3\envs\tensorflow\lib\site-packages\keras\models.py:844: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 239680 samples, validate on 79894 samples
Epoch 1/20


ResourceExhaustedError: OOM when allocating tensor with shape[183292,300] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: mul_4 = Mul[T=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:GPU:0"](beta_1/read, Variable/read)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: mul_2/_79 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_6861_mul_2", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.


Caused by op 'mul_4', defined at:
  File "C:\Users\RAKSHITH_RAO\Anaconda3\envs\tensorflow\lib\runp

In [ ]:
print("Evaluation...")
loss, acc = model.evaluate([Xqtest, Xatest], Ytest, batch_size=BATCH_SIZE)
print("Test loss/accuracy final model = %.4f, %.4f" % (loss, acc))

model.save_weights(os.path.join(MODEL_DIR, "qa-blstm-fem-attn-final.hdf5"))
with open(os.path.join(MODEL_DIR, "qa-blstm-fem-attn.json"), "wb") as fjson:
    fjson.write(model.to_json().encode('utf8'))

model.load_weights(filepath=os.path.join(MODEL_DIR, "qa-blstm-fem-attn-best.hdf5"))
loss, acc = model.evaluate([Xqtest, Xatest], Ytest, batch_size=BATCH_SIZE)
print("\nTest loss/accuracy best model = %.4f, %.4f" % (loss, acc))

In [ ]:
qapairs = kaggle.get_question_answer_pairs(
      question_file = os.path.join(DATA_DIR, QA_TRAIN_FILE),qaf_file=os.path.join(DATA_DIR, FLASHCARD_SENTS) )
tqapairs = kaggle.get_question_answer_pairs(
    question_file = os.path.join(DATA_DIR, "Middle-NDMC-Test.csv"), is_test=True,qaf_file='')
#qavec = Word2Vec.load(os.path.join(DATA_DIR, qa_BIN))

In [ ]:
#### read in the data ####
#### correct_answer = "B"
question = "Which is a distinction between an epidemic and a pandemic?"
answers = ["the symptoms of the disease",
           "the geographical area affected",
           "the species of organisms infected",
           "the season in which the disease spreads"]
qwords = nltk.word_tokenize(question)
awords_list = [nltk.word_tokenize(answer) for answer in answers]
#for awords in awords_list:
#    tqapairs.append((qwords, awords, None))
Xq, Xa = [], []

In [ ]:
word2idx = kaggle.build_vocab([], qapairs, tqapairs)
vocab_size = len(word2idx) + 1 # include mask character 0

In [ ]:
for idx, awords in enumerate(awords_list):
    Xq.append([word2idx[qword] for qword in qwords])
    Xa.append([word2idx[aword] for aword in awords])
Xq = pad_sequences(Xq, maxlen=seq_maxlen)
Xa = pad_sequences(Xa, maxlen=seq_maxlen)

In [ ]:
Y = model.predict([Xq,Xa])

# calculate the softmax
probs = np.exp(1.0 - (Y[:, 1] - Y[:, 0]))
probs = probs / np.sum(probs)

print(probs)

In [ ]:
plt.bar(np.arange(len(probs)), probs)
plt.xticks(np.arange(len(probs))+0.35, ["A", "B", "C", "D"])
plt.xlabel("choice (x)")
plt.ylabel("probability p(x)")
plt.show()